##Stream Customer Data using Auto Loader from cloud files to Delta Location
<ol>
<li> Read files from cloud storage using Data Reader API Auto Loader</li>
<p><li> Transform dataframe to add following columns</li>
              <ul>
              <li>file_path</li>
              <li>ingestion date:Current timestamp</li>
              </ul>
</p>
<li> Write Transform Data Stream to Delta Lake Table</li>
</ol>

###1.Read files from cloud storage using Data Reader API Autoloader

In [0]:
customer_df=spark.readStream.format('cloudFiles')\
  .option('cloudFiles.format','json')\
  .option('cloudFiles.schemaLocation','abfss://gizmobox@sambitdatabricksstorage.dfs.core.windows.net/landing/customers_autoloader/_schema')\
  .option('cloudFiles.useNotifications','true')\
  .option('cloudFiles.inferColumnTypes','true')\
  .option('cloudFiles.schemaEvolutionMode','addNewColumns')\
  .option('cloudFiles.rescuedDataColumn','rescued_data')\
  .load('/Volumes/gizmobox_stream_cat/landing/customer_autoloader/')

In [0]:
customer_df.display()

###2.Transform dataframe to add following columns

In [0]:
from pyspark.sql.functions import *

In [0]:
customer_transformed_df=customer_df.withColumn('filepath',col('_metadata.file_path'))\
    .withColumn('ingestion_date',current_timestamp())

###3.Write Transform Data Stream to Delta Lake Table

In [0]:
streaming_query=customer_transformed_df.writeStream.format('delta')\
  .outputMode('append')\
  .trigger(processingTime="2 minutes")\
  .option('checkpointLocation','/Volumes/gizmobox_stream_cat/landing/customer_stream/_checkpoint_stream').toTable('gizmobox_stream_cat.bronze.customers_stream')

In [0]:
streaming_query.stop()

In [0]:
%sql
select * from gizmobox_stream_cat.bronze.customers_stream;
-- this select statement is not running as streaming query but .writeStream is working on streaming query